# ¿Cuáles es la región con mayor potencial de ganancias?
---
La industria petrolera OilyGiant quiere saber los mejores 200 lugares para abrir pozos de extracción.  
- El objetivo de este proyecto es diseñar un modelo predictivo eficiente que nos proporcione la información necesria para predecir las zonas de extracción que podrían tener las más altas remuneraciones con un riesgo de error suficientemente bajo. Utilizaremos la técnica de bootstrapping para calcular el riesgo con un intervalo de confianza del 95%.

## Análisis y limpieza de datos

In [ ]:
# Importar Pandas y Cargar datos
import pandas as pd
import numpy as np
df_0 = pd.read_csv('/datasets/geo_data_0.csv')
df_1 = pd.read_csv('/datasets/geo_data_1.csv')
df_2 = pd.read_csv('/datasets/geo_data_2.csv')

### Examinación y análisis datos

In [ ]:
# Examinar datos
print(df_0.info())
print()
print(df_1.info())
print()
print(df_2.info())
print()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column

In [ ]:
# Estudiar datos
print('Región 1:')
print(df_0.sample(3))
print()
print('Región 2:')
print(df_1.sample(3))
print()
print('Región 3:')
print(df_2.sample(3))
print()

Región 1:
          id        f0        f1        f2     product
16193  x29US -0.077578  0.190152  6.438142  109.358195
4077   uMT91  1.866091  0.183872 -0.427140   99.199093
85803  XBGPU  0.369470 -0.298553  0.037431   32.346583

Región 2:
          id        f0        f1        f2    product
49724  e4EKC -8.584535 -5.430919  0.998738  30.132364
86475  e8LvC -4.912259 -3.689451  2.997680  84.038886
58541  xOTFi  4.909936 -5.418065  1.005010  30.132364

Región 3:
          id        f0        f1        f2     product
80974  tfccY -0.022556 -2.084414  2.008469  107.960421
25021  igV7N  1.516232 -3.200092  3.418939   81.300058
88231  565pX -0.143192 -0.663331  1.661260  105.239730



In [14]:
df_0.duplicated().sum()

0

In [15]:
df_1.duplicated().sum()

0

In [16]:
df_2.duplicated().sum()

0

#### Conclusión 
Las bases de datos están estructurados de la misma manera y no necesitan ser formateados.

## Predicción de volúmen promedio de reservas y RMSE

In [ ]:
# Creación de función
def prediction_valid_rmse_mean(df):
    # Cargamos librerias necesarias
    from sklearn.model_selection import train_test_split
    from sklearn.linear_model import LinearRegression
    from sklearn.metrics import mean_squared_error

    # Seleccionamos features y target
    features = df.drop(['id', 'product'], axis=1)
    target = df['product']

    # Dividimos base de datos 75-25
    random = 23451
    features_train, features_valid, target_train, target_valid = train_test_split(
        features, target, test_size=0.25, random_state=random)

    # Guardar indices
    index_valid = target_valid.index
    ids_valid = df.loc[index_valid, 'id']

    # Entrenamos modelo y predecimos resultados
    model = LinearRegression()
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    rmse = (mean_squared_error(target_valid, predictions_valid))**0.5
    mean_reserves = predictions_valid.mean()

   # Guardamos resultados en un dataframe del conjunto de validación
    df_prediction_valid = pd.DataFrame({
        'id': ids_valid,
        'predictions': predictions_valid,
        'real_value': target_valid,
        'difference': predictions_valid - target_valid.values})

    return df_prediction_valid, rmse, mean_reserves

### Mostramos los resultados por región

In [ ]:
# Usamos la función de regresión lineal y guardamos resultados
predictions_valid_df_r1, rmse_r1, mean_reserves_r1 = prediction_valid_rmse_mean(
    df_0)
predictions_valid_df_r2, rmse_r2, mean_reserves_r2 = prediction_valid_rmse_mean(
    df_1)
predictions_valid_df_r3, rmse_r3, mean_reserves_r3 = prediction_valid_rmse_mean(
    df_2)

In [ ]:
# Imprimimos todos los resultados
print(f'Los resultados de la región 1 son:')
print(
    f'El RECM del modelo de regresión lineal en el conjunto de validación predice un alejamiento aproximado de {rmse_r1.round(2)} mil barriles del valor real.')
print(
    f'El volúmen promedio de reservas predicho es de: {mean_reserves_r1.round(2)} mil barriles.\n')
print(f'Los resultados de la región 2 son:')
print(
    f'El RECM del modelo de regresión lineal en el conjunto de validación predice un alejamiento aproximado de {rmse_r2.round(2)} mil barriles del valor real.')
print(
    f'El volúmen promedio de reservas predicho es de: {mean_reserves_r2.round(2)} mil barriles.\n')
print(f'Los resultados de la región 3 son:')
print(
    f'El RECM del modelo de regresión lineal en el conjunto de validación predice un alejamiento aproximado de {rmse_r3.round(2)} mil barriles del valor real.')
print(
    f'El volúmen promedio de reservas predicho es de: {mean_reserves_r3.round(2)} mil barriles.\n')

Los resultados de la región 1 son:
El RECM del modelo de regresión lineal en el conjunto de validación predice un alejamiento aproximado de 37.75 mil barriles del valor real.
El volúmen promedio de reservas predicho es de: 92.48 mil barriles.

Los resultados de la región 2 son:
El RECM del modelo de regresión lineal en el conjunto de validación predice un alejamiento aproximado de 0.88 mil barriles del valor real.
El volúmen promedio de reservas predicho es de: 69.05 mil barriles.

Los resultados de la región 3 son:
El RECM del modelo de regresión lineal en el conjunto de validación predice un alejamiento aproximado de 40.13 mil barriles del valor real.
El volúmen promedio de reservas predicho es de: 94.77 mil barriles.



## Preparación de datos para el cálculo de ganancias

In [ ]:
budget = 100000000
choosen_wells = 200
break_even_point = 500000
break_even_point_barrel = 111.1
price_per_barrel = 500000 / 111.1

### Conclusión
Ningún región alcanza el volúmen de reserva promedio mínimo de 111.1 barriles, esto nos indica que tenemos que seleccionar los mejores pozos y que no podemos dejar la selección al azar.  
Necesitamos saber nuestro punto de equilibrio para no perder dinero, en este caso invertiremos 100 millones de dólares para extraer 200 pozos. Necesitamos extraer por lo menos 111.1 barriles de petróleo o lo que es lo mismo 500 mil dólares por pozo para evitar pérdidas.  
Ahora, con nuestro modelo debemos elegir los 200 pozos con mejor predicción de cada región. Después de esto necesitamos descubrir:
- Que cada pozo produzca por lo menos su punto de equilibrio (500 mil dólares).

## Predicción de ganancias potenciales por región

In [ ]:
# Creación de función para todo el dataframe
def full_df_prediction(df):
    # Cargamos librerias necesarias
    from sklearn.model_selection import train_test_split
    from sklearn.linear_model import LinearRegression

    # Seleccionamos features y target
    features = df.drop(['id', 'product'], axis=1)
    target = df['product']

    # Dividimos base de datos 75-25
    random = 23451
    features_train, features_valid, target_train, target_valid = train_test_split(
        features, target, test_size=0.25, random_state=random)

    # Hacemos la predicción de todo el dataframe
    model = LinearRegression()
    model.fit(features_train, target_train)
    predictions_df = model.predict(features)

    # Guardamos resultados en un dataframe COMPLETO
    df_prediction = pd.DataFrame({
        'id': df['id'],
        'predictions': predictions_df,
        'real_value': target,
        'difference': target.values - predictions_df})

    # Seleccionamos los 200 mejores pozos
    df_prediction_better_200_wells = df_prediction.sort_values(
        by='predictions', ascending=False).head(choosen_wells)

    # Calculamos la ganancia potencial
    potencial_profits = ((((df_prediction_better_200_wells['predictions'].sum(
    )) * price_per_barrel)) - budget).round(2)

    return df_prediction, df_prediction_better_200_wells, potencial_profits

In [ ]:
# Mostramos las ganancias potenciales por región
predictions_df_r1, better_200_wells_r1, potencial_profits_r1 = full_df_prediction(
    df_0)
predictions_df_r2, better_200_wells_r2, potencial_profits_r2 = full_df_prediction(
    df_1)
predictions_df_r3, better_200_wells_r3, potencial_profits_r3 = full_df_prediction(
    df_2)

print(potencial_profits_r1)
print(potencial_profits_r2)
print(potencial_profits_r3)
print()

46869954.67
25257448.37
41029221.65



### Conclusión
La región 1 predice una ganancia de **\$46,869.95** millones de dólares.  
La región 2 predice una ganancia de **\$25,257.45** millones de dólares.  
La región 3 predice una ganancia de **\$41,029.22** millones de dólares.  

Hasta ahora la **región 1** parece ser la mejor por las siguientes razones:
- Tiene el mayor potencial de ganancia con \$46,869 millones de dólares.
- Nuestro modelo arrojo un RMSE aceptable con un error aproximado de 37.75 barriles alejado del promedio.
- El volúmen de reservbas predicho es el segundo más alto con 92.48 barriles.  
*Nota: La elección puede cambiar más adelante con otros análisis.*





## Cálculo de riesgos y ganancias por región

In [ ]:
# Cramos función para realizar bootstrapping
def real_profit_per_well(best_200, break_even_point=500000, price_per_barrel=(500000 / 111.1), n_samples=1000):
    best_200['real_profit_per_well'] = (
        best_200['real_value'] * price_per_barrel) - break_even_point

    profits = []
    for i in range(n_samples):
        sample = np.random.choice(
            best_200['real_profit_per_well'], replace=True)
        profits.append(sample.round(2))

    return profits, ((sum(profits)) / len(profits)).round(2), best_200.sort_values(by='real_profit_per_well', ascending=False), best_200['real_profit_per_well'].sum()

In [ ]:
# Utilizamos función y presentamos resultados
profits_r1, mean_profit_r1, df_profits_r1, total_profit_r1 = real_profit_per_well(
    better_200_wells_r1)
profits_r2, mean_profit_r2, df_profits_r2, total_profit_r2 = real_profit_per_well(
    better_200_wells_r2)
profits_r3, mean_profit_r3, df_profits_r3, total_profit_r3 = real_profit_per_well(
    better_200_wells_r3)

print(f'Los resultados de la región 1 son:')
print(f' -Beneficio promedio real: {mean_profit_r1},')
print(
    f' -Intervalo de confianza del 5% igual a: {np.percentile(profits_r1, [2.5, 97.5]).round(2)},')
print(
    f' -Riesgo equivalente a: {((np.array(profits_r1) < 0).mean() * 100).round(2)}%.')

print(f'Los resultados de la región 2 son:')
print(f' -Beneficio promedio real: {mean_profit_r2},')
print(
    f' -Intervalo de confianza del 5% igual a: {np.percentile(profits_r2, [2.5, 97.5]).round(2)},')
print(
    f' -Riesgo equivalente a: {((np.array(profits_r2) < 0).mean() * 100).round(2)}%.')

print(f'Los resultados de la región 3 son:')
print(f' -Beneficio promedio real: {mean_profit_r3},')
print(
    f' -Intervalo de confianza del 5% igual a: {np.percentile(profits_r3, [2.5, 97.5]).round(2)},')
print(
    f' -Riesgo equivalente a: {((np.array(profits_r3) < 0).mean() * 100).round(2)}%.')

Los resultados de la región 1 son:
 -Beneficio promedio real: 180151.15,
 -Intervalo de confianza del 5% igual a: [-46294.64 320131.59],
 -Riesgo equivalente a: 4.4%.
Los resultados de la región 2 son:
 -Beneficio promedio real: 120816.42,
 -Intervalo de confianza del 5% igual a: [120816.42 120816.42],
 -Riesgo equivalente a: 0.0%.
Los resultados de la región 3 son:
 -Beneficio promedio real: 123840.94,
 -Intervalo de confianza del 5% igual a: [-180574.35  319707.71],
 -Riesgo equivalente a: 17.1%.


### Conclusión
La región 1 tuvo una ganancia real promedio de **\$180,151.15** dólares por pozo, con un 4.4\% de riesgo de generar pérdida.  
La región 2 tuvo una ganancia real promedio de **\$120,816.42** dólares por pozo, con un 0.0\% de riesgo de generar pérdida.  
La región 3 tuvo una ganancia real promedio de **\$123,840.94,** dólares por pozo, con un 17.1\% de riesgo de generar pérdida.

¿Qué decisión tomar?  
En esta situación debemos elegir entre obtener \$120,816.42 dólares por pozo **sin** riesgo de pérdida o aceptar un riesgo de pérdida de 4.4\% por pozo pero con un promedio de ganancia mayor por pozo con \$180,151.15.


**Elección final**  
Recomiendo ampliamente elegir por segunda ocasión la **región 1**, por un riesgo de pérdida por pozo de tan solo 4.4\% accedemos a un margen de ganancia mayor aproximado al 50\% comparado a la región 2. La región 3 está completamente descartada ya que el riesgo de seleccionar un pozo con pérdida es del 17.1\% y el promedio de ganancia casi identico a la región 2.